# 🔌 Problem 1: Equivalent Resistance Using Graph Theory

<div style="background-color: #f0f8ff; padding: 15px; border-radius: 10px;">
<h2 style="color: #2E86C1; text-align: center;">📐 Visualizing Circuit Simplification Step-by-Step</h2>
</div>

---

## 🎯 Motivation

Calculating equivalent resistance is a fundamental task in circuit analysis. While series and parallel rules work well for simple circuits, **graph theory** provides a powerful, automated way to analyze **complex resistor networks**.

A circuit is modeled as a graph:
- **Nodes** represent junctions
- **Edges** represent resistors (with weights equal to resistance)

---

## 🧠 Approach

We reduce the graph step by step using:

1. 🔗 **Series Reduction**  
   Two resistors in a chain become one with total resistance:  
   $$ R_{\text{eq}} = R_1 + R_2 $$

2. 🔁 **Parallel Reduction**  
   Two or more resistors between the same nodes:  
   $$ \frac{1}{R_{\text{eq}}} = \frac{1}{R_1} + \frac{1}{R_2} + \cdots $$

3. 🔄 Repeat until the entire network reduces to a single equivalent resistance.

---

## 🧮 Example Circuit

A simple 4-node graph with resistors:

- 2Ω between nodes (1)-(2)  
- 3Ω between nodes (2)-(3)  
- 4Ω between nodes (3)-(4)  
- 1Ω directly between nodes (1)-(4)

---

## ⚙️ Algorithm Pseudocode

```plaintext
while graph not reduced:
    if a node has exactly 2 neighbors:
        combine series resistors
    if parallel resistors exist:
        combine using reciprocal rule


In [14]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import os

# Define the circuit as a graph
G = nx.Graph()
G.add_edge(1, 2, resistance=2)
G.add_edge(2, 3, resistance=3)
G.add_edge(3, 4, resistance=4)
G.add_edge(1, 4, resistance=1)

# Prepare frame images for GIF
frame_images = []

# Function to draw and capture frame (uses FigureCanvas explicitly)
def capture_graph_frame(graph):
    fig, ax = plt.subplots(figsize=(6, 5))
    canvas = FigureCanvas(fig)
    pos = nx.spring_layout(graph, seed=42)
    labels = {e: f"{d['resistance']}Ω" for e, d in graph.edges.items()}
    nx.draw(graph, pos, with_labels=True, ax=ax, node_size=600, node_color='lightblue', edge_color='gray')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels, ax=ax)
    canvas.draw()
    frame = np.frombuffer(canvas.buffer_rgba(), dtype='uint8')
    frame = frame.reshape(canvas.get_width_height()[::-1] + (4,))
    frame_images.append(frame)
    plt.close(fig)

# Series combination logic
def combine_series(graph):
    for node in list(graph.nodes()):
        if graph.degree[node] == 2:
            u, v = list(graph.neighbors(node))
            if graph.has_edge(u, node) and graph.has_edge(node, v):
                r1 = graph[u][node]['resistance']
                r2 = graph[node][v]['resistance']
                graph.remove_node(node)
                graph.add_edge(u, v, resistance=r1 + r2)
                return True
    return False

# Placeholder for parallel logic
def combine_parallel(graph):
    return False

# Run the simplification and capture each step
def simplify_and_capture(graph):
    G_copy = graph.copy()
    while True:
        capture_graph_frame(G_copy)
        if not (combine_series(G_copy) or combine_parallel(G_copy)):
            break
    return G_copy

# Run
G_final = simplify_and_capture(G)

# Create the animation (slower timing)
fig, ax = plt.subplots()
im = ax.imshow(frame_images[0])

def update(i):
    im.set_data(frame_images[i])
    return [im]

ani = FuncAnimation(fig, update, frames=len(frame_images), interval=2000)  # 2 seconds per frame
gif_name = "equivalent_resistance_simplification.gif"
ani.save(gif_name, writer=PillowWriter(fps=0.5))  # slower FPS

plt.close()

# Auto-open the saved GIF
try:
    os.startfile(gif_name)
except:
    print(f"Saved: {gif_name}. Please open manually.")
